In [1]:
import os

In [2]:
pwd

'c:\\Prajesh\\personal\\ML_learning\\my_work\\kubeflow_app\\kubeflow_ML_app\\ML_flow_app\\research'

In [3]:
cd ..

c:\Prajesh\personal\ML_learning\my_work\kubeflow_app\kubeflow_ML_app\ML_flow_app


In [4]:
os.environ["MLFLOW_TRACKING_URI"]='http://ec2-43-205-243-196.ap-south-1.compute.amazonaws.com:5000/'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelEvaluationConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: str
    target_col: str
    mlflow_uri: str

In [6]:
from mlFlowProject.constants import *
from mlFlowProject.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params.RandomForestRegressor
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_col=schema.name,
            mlflow_uri = config.mlflow_uri
        )
        
        return model_evaluation_config

In [15]:
# dummy = ConfigurationManager()
# dummy.get_model_evaluation_config()

In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score
import os
import pandas as pd
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
import numpy as np
import joblib
from mlFlowProject.utils.common import save_json
from pathlib import Path

In [27]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def evaluate_model(self, ytrue, ypred):
        rmse = np.sqrt(mean_squared_error(ytrue, ypred))
        mae = mean_absolute_error(ytrue, ypred)
        r2 = r2_score(ytrue, ypred)
        
        train_data = pd.read_csv(self.config.train_data_path)
        xtrain = np.array(train_data['Open']).reshape(-1, 1)
        ytrain = np.array(train_data[[self.config.target_col]]).reshape(-1)
        
        model = joblib.load(self.config.model_path)
        
        cv_score = cross_val_score(model, xtrain, ytrain, cv = 5)
        return rmse, mae, r2, cv_score
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        xtest = np.array(test_data['Open']).reshape(-1, 1)
        ytest = np.array(test_data[[self.config.target_col]]).reshape(-1)
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            
            ypred = model.predict(xtest)
            
            (rmse, mae, r2, cv_score) = self.evaluate_model(ytest, ypred)
            
            cv_score = cv_score.tolist()
            
            scores = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2,
                "mean_cv_accuracy": np.mean(cv_score),
                "cv_accuracy_std": np.std(cv_score)
            }
            
            save_json(path = Path(self.config.metric_file_name), data = scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mean_cv_accuracy", np.mean(cv_score))
            mlflow.log_metric("cv_accuracy_std", np.std(cv_score))
            
            # if tracking_url_type_store != "file":
            #     mlflow.sklearn.log_model(model, "model", registered_model_name = "RandomForestRegressor")
            # else:
            mlflow.sklearn.log_model(model, "model")
        
        

In [28]:
try:
    config_manager = ConfigurationManager()
    model_eval_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_eval_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-08-06 15:04:49,461: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-06 15:04:49,463: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-06 15:04:49,466: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-06 15:04:49,467: INFO: common: created directory at: artifacts]
[2023-08-06 15:04:49,468: INFO: common: created directory at: artifacts/model_evaluation]
[2023-08-06 15:04:49,509: DEBUG: connectionpool: Resetting dropped connection: ec2-43-205-243-196.ap-south-1.compute.amazonaws.com]
[2023-08-06 15:04:49,546: DEBUG: connectionpool: http://ec2-43-205-243-196.ap-south-1.compute.amazonaws.com:5000 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 935]
[2023-08-06 15:04:49,801: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
[2023-08-06 15:04:49,812: DEBUG: connectionpool: Resetting dropped connection: ec2-43-205-243-196.ap-south-1.compute.amazonaws.com]
[2023-08-06 15:04:49,996: DEBUG: connectionpool